## 2. A, C and Z frequency weightings

After we apply compensation to the input signal by means of the designed inverse filter, the next step is to pass the output audio to the **frequency weighting network**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (15,9)
sns.set_context('notebook', font_scale=1.2)